
# Analysis Comparison between Original and Expanded Lexicons on Cohort Analysis Task

Subreddit: BPD

In [1]:
from pathlib import Path
import pandas as pd

from config import global_config
from lexicons2 import Values, ValuesExpanded, Liwc2015, Liwc2015Expanded
from spaces import WordEmbeddings, LabelEmbeddings

In [2]:
COHORT_NAME = 'BPD'

In [3]:
cohort_corpus_path = global_config.reddit.submissions[COHORT_NAME]
cohort_models_path = f'{global_config.paths.models}/{COHORT_NAME}'

control_corpus_path = global_config.reddit.submissions['IAMA']
control_models_path = f'{global_config.paths.models}/IAMA'

In [4]:
cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks.csv'
expanded_cohort_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_ranks_expanded.csv'

control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks.csv'
expanded_control_ranks_csv = f'{global_config.paths.ranks}/IAMA_ranks_expanded.csv'

relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks.csv'
expanded_relative_ranks_csv = f'{global_config.paths.ranks}/{COHORT_NAME}_relative-ranks_expanded.csv'

Build Control Spaces

In [5]:
control_word_space = WordEmbeddings(corpus_path=control_corpus_path, model_path=control_models_path).build()

In [6]:
org_control_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=control_word_space).build()

In [7]:
expanded_control_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=control_word_space).build()

Build Cohort Spaces

In [8]:
cohort_word_space = WordEmbeddings(corpus_path=cohort_corpus_path, model_path=cohort_models_path).build()

In [9]:
org_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015(), Values()], word_embeddings=cohort_word_space).build()

In [10]:
expanded_cohort_label_space = LabelEmbeddings(lexicons=[Liwc2015Expanded(), ValuesExpanded()], word_embeddings=cohort_word_space).build()

Save and view results

In [11]:
def save_if_not_save_and_return(space: LabelEmbeddings, path: str):
	if not Path(path).exists():
		space.save_distances_to_csv(path)
	return pd.read_csv(path, names=['label_one', 'label_two', 'distance'])

control_ranks = save_if_not_save_and_return(org_control_label_space, control_ranks_csv)
expanded_control_ranks = save_if_not_save_and_return(expanded_control_label_space, expanded_control_ranks_csv)

cohort_ranks = save_if_not_save_and_return(org_cohort_label_space, cohort_ranks_csv)
expanded_cohort_ranks = save_if_not_save_and_return(expanded_cohort_label_space, expanded_cohort_ranks_csv)

In [12]:
org_cohort_label_space.compute_rank_deltas(org_control_label_space, relative_ranks_csv)
relative_ranks = pd.read_csv(relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

expanded_cohort_label_space.compute_rank_deltas(expanded_control_label_space, expanded_relative_ranks_csv)
expanded_relative_ranks = pd.read_csv(expanded_relative_ranks_csv, names=['label_one', 'label_two', 'current_rank', 'control_rank', 'current_distance', 'control_distance', 'rank_delta', 'distance_delta'])

View Stuff

In [13]:
relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,values:learning,70,28,0.241659,0.178845,42,0.062815
1,liwc2015:function,values:feeling-good,73,43,0.293116,0.499940,30,-0.206824
2,liwc2015:function,liwc2015:work,59,36,0.114591,0.336770,23,-0.222178
3,liwc2015:function,liwc2015:money,71,51,0.246038,0.681783,20,-0.435746
4,liwc2015:function,values:animals,82,64,1.036170,0.794112,18,0.242059
...,...,...,...,...,...,...,...,...
6967,values:animals,liwc2015:assent,1,63,0.883170,0.948860,-62,-0.065689
6968,values:animals,values:children,8,76,0.979765,1.028190,-68,-0.048425
6969,values:animals,liwc2015:sexual,3,75,0.906185,1.021016,-72,-0.114831
6970,values:animals,values:order,7,79,0.978004,1.078921,-72,-0.100918


In [14]:
expanded_relative_ranks

,label_one,label_two,current_rank,control_rank,current_distance,control_distance,rank_delta,distance_delta
0,liwc2015:function,liwc2015:work,68,39,0.114591,0.336770,29,-0.222178
1,liwc2015:function,liwc2015:posemo,51,34,0.030827,0.259349,17,-0.228522
2,liwc2015:function,liwc2015:money,75,58,0.246038,0.681783,17,-0.435746
3,liwc2015:function,liwc2015:we,41,25,0.013967,0.130866,16,-0.116898
4,liwc2015:function,liwc2015:discrep,39,26,0.012835,0.131881,13,-0.119046
...,...,...,...,...,...,...,...,...
7135,liwc2015:ingest,liwc2015:tentat,3,54,0.256178,1.014663,-51,-0.758485
7136,liwc2015:ingest,liwc2015:shehe,21,78,0.261032,1.089677,-57,-0.828645
7137,liwc2015:ingest,liwc2015:compare,1,61,0.246522,1.033294,-60,-0.786771
7138,liwc2015:ingest,liwc2015:adj,0,60,0.245306,1.032501,-60,-0.787195
